## Hyperopt is a Hyperparamter Optimization Library

It Minimizes a function f(x1,x2....,xn) when (x1,x2....,xn) belong to some Range

In [1]:
from hyperopt import hp, tpe, fmin
# hp is the function which will create a range for the function parameters
# tpe is the Tree-structured Parzen Estimator Algorithm, which we use instead of Grid/Random search as dicussed
# fmin is the function that will take in the function that is to be optimized, the range of parameters

In [2]:
def objective_function(args):  # define the function which will take the argument 
                               # and return the equation that is to be minimized.
    x,y = args['x'],args['y']  # taking the arguments
    return x**2 - y**2         # returning the equation

In [3]:
space = {'x':hp.uniform('x',-2,2),      #defining the range for x
         'y':hp.uniform('y',-3,1)}      #defining the range for y

In [13]:
best_param_20eval = fmin(objective_function ,space ,algo=tpe.suggest ,max_evals=60)
# using tpe to get the minimum value of f(x,y) when x <- (-2,2)
#                                               and y <- (-3,1) 

In [14]:
print(best_param_20eval)

{'y': -2.9970869796100037, 'x': 0.5489189881566032}


In [16]:
#almost similar result for 100 evals
best_param_100eval = fmin(objective_function ,space ,algo=tpe.suggest ,max_evals=500)
print(best_param_100eval)

{'y': -2.998779147754817, 'x': 0.0073462322186383655}


## Understanding and Creating Search Space for hyperopt
Unlike sklearn's grid search hyperopt.fmin's search space must be an hp object. We'll look at some important range functions and how to use them

Fmin function allow us to pass only one argument to objective function as can be seen in above example, the we are supposed to extract the parameters from the argument.

we can use the above code like this as well:

```
def objective_function(args):
    x,y = args[0],args[1]  
    return x**2 - y**2
    
space = [hp.uniform('x',-2,2),hp.uniform('y',-3,1)]
#or
space = ([hp.uniform('x',-2,2),hp.uniform('y',-3,1)])

```

### Lets look at some hyperopt space functions on the official wiki page here:
https://github.com/hyperopt/hyperopt/wiki/FMin#21-parameter-expressions

## We'll now use some complex spaces in the following sklearn examples and Compare Grid Search and tpe for optimization of 

In [25]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd

df = pd.read_csv('iris.csv')

In [26]:
df.head()  #our data

,sepal length,sepal width,petal length,petal width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [27]:
l = LabelEncoder()
df['species'] = l.fit_transform(df['species'])

x = df.drop('species',axis=1)
y = df['species']

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

### Using Grid Search

In [28]:
space = {'n_neighbors':hp.choice('n_neighbors',range(3,11)),
        'algorithm':hp.choice('algorithm',['ball_tree','kd_tree']),
        'leaf_size':hp.choice('leaf_size',range(1,50)),
        'metric':hp.choice('metric',["euclidean","manhattan","chebyshev","minkowski","wminkowski","seuclidean","mahalanobis"]),
        }

# so we have a lots of possiblities for KNearestClassifier to look for the best set of HyperParameter

def find_hp_knn(args):
    n_neighbors = args['n_neighbors']
    algorithm = args['algorithm']
    leaf_size = args['leaf_size']
    metric = args['metric']
    
    model = KNeighborsClassifier(n_neighbors=n_neighbors,algorithm=algorithm,leaf_size=leaf_size,metric=metric)
    
    model.fit(x_train,y_train)
    
    y_pred_train = model.predict(x_train)
    
    loss = mean_squared_error(y_train,y_pred_train)
    
    
    
    